In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.graph_objects as go
import plotly.express as px
import os
import plotly.io as plt
from pretty_html_table import build_table
# Enable this to export as iframe
# plt_io.renderers.default = 'iframe'
pd.options.mode.chained_assignment = None

In [ ]:
# Global variables
default_font_family = 'IBM Plex Sans'
default_font_size = 12
colors = ['#E7C65B', '#225560', '#310D20', '#96031A']
logo_path = os.path.join(os.getcwd(), 'INPUT/sip-logo.png')
DEBUG = False

# 1. Data Preprocessing

In [ ]:
trello_table = pd.read_csv('./INPUT/trello_board.csv')
trello_fields_dict = {
    'Card Name': 'T#',
    'Card Description': 'DESC',
    'Labels': 'LABELS',
    'List Name': 'STATUS',
    'T_CRE_TS': 'TICKET_CREATION_TIMESTAMP',
    'Card ID': 'TICKET_RESPONSE_TIMESTAMP',
    'T_RES_TS': 'TICKET_RESOLUTION_TIMESTAMP',
    }
trello_table = trello_table[trello_fields_dict]
trello_table.rename(columns=trello_fields_dict, inplace=True)

# Only analyze resolved tickets
trello_table = trello_table[trello_table['STATUS'] == 'RESOLVED']
trello_table = trello_table.sort_values(by=['T#'], ignore_index=True)

if DEBUG:
    trello_table[trello_table.isna().any(axis=1)]

# Convert timestamps datetime objects
trello_table['TICKET_CREATION_TIMESTAMP'] = \
    pd.to_datetime(trello_table['TICKET_CREATION_TIMESTAMP'])
trello_table['TICKET_RESOLUTION_TIMESTAMP'] = \
    pd.to_datetime(trello_table['TICKET_RESOLUTION_TIMESTAMP'])

# Generate start and end date of the trello table
trello_table.start_timestamp = trello_table['TICKET_CREATION_TIMESTAMP'
        ].min().strftime('%d/%m/%Y|%H:%M:%S')
trello_table.end_timestamp = trello_table['TICKET_CREATION_TIMESTAMP'
        ].max().strftime('%d/%m/%Y|%H:%M:%S')
trello_table.no_days = len(trello_table['TICKET_CREATION_TIMESTAMP'
                           ].dt.normalize().unique())
if DEBUG:
    # Check if we have error in timestamps
    trello_table['ERROR'] = (trello_table['TICKET_CREATION_TIMESTAMP'
                             ].dt.second
                             >= trello_table['TICKET_RESOLUTION_TIMESTAMP'
                             ].dt.second).astype(int)

In [ ]:

# To get the card creation timestamp from it's ID
trello_table['TICKET_RESPONSE_TIMESTAMP'] = [x[:8] for x in trello_table['TICKET_RESPONSE_TIMESTAMP']]
trello_table['TICKET_RESPONSE_TIMESTAMP'] = trello_table['TICKET_RESPONSE_TIMESTAMP'].apply(int, base=16)
trello_table['TICKET_RESPONSE_TIMESTAMP'] = pd.to_datetime(trello_table['TICKET_RESPONSE_TIMESTAMP'],unit='s')
trello_table['TICKET_RESPONSE_TIMESTAMP'] = trello_table['TICKET_RESPONSE_TIMESTAMP'].dt.tz_localize('GMT').dt.tz_convert('Asia/Riyadh').dt.tz_localize(None)

trello_table['TICKET_CREATION_TIMESTAMP_ADJUSTED'] = trello_table['TICKET_CREATION_TIMESTAMP']
trello_table['TICKET_RESPONSE_TIMESTAMP_ADJUSTED'] = trello_table['TICKET_RESPONSE_TIMESTAMP']
trello_table['TICKET_RESOLUTION_TIMESTAMP_ADJUSTED'] = trello_table['TICKET_RESOLUTION_TIMESTAMP']
if DEBUG:
    display(trello_table)


In [ ]:
sip_bh = pd.offsets.CustomBusinessHour(start='07:30', end='15:30', weekmask='Sun Mon Tue Wed Thu')

for index, row in trello_table.iterrows():
    roll1 = sip_bh.rollforward(pd.Timestamp(trello_table['TICKET_CREATION_TIMESTAMP'][index]))
    trello_table['TICKET_CREATION_TIMESTAMP_ADJUSTED'][index] = roll1
    roll2 = sip_bh.rollforward(pd.Timestamp(trello_table['TICKET_RESPONSE_TIMESTAMP'][index]))
    trello_table['TICKET_RESPONSE_TIMESTAMP_ADJUSTED'][index] = roll2
    roll3 = sip_bh.rollforward(pd.Timestamp(trello_table['TICKET_RESOLUTION_TIMESTAMP'][index]))
    trello_table['TICKET_RESOLUTION_TIMESTAMP_ADJUSTED'][index] = roll3


trello_table['BUSINSESS_HOURS_TO_ACKNOWLEDGE'] = trello_table.apply(lambda x: len(pd.date_range(start=x.TICKET_CREATION_TIMESTAMP, end=x.TICKET_RESPONSE_TIMESTAMP, freq= sip_bh)),axis=1)
trello_table['BUSINSESS_HOURS_TO_RESOLVE'] = trello_table.apply(lambda x: len(pd.date_range(start=x.TICKET_CREATION_TIMESTAMP, end=x.TICKET_RESOLUTION_TIMESTAMP, freq= sip_bh)),axis=1)

In [ ]:
# Handle labels
labels_dict = {'SC' : 'SEC_CONTROL', 'RC' : 'RES_CODE', 'CO' : 'DEPENDENT_ON', 'TB' : 'ROOTICKET_CAUSE', 'PR' : 'PRIORITY'}
trello_table["LABELS"] = trello_table["LABELS"].str.replace('\([^)]*\)', '', regex=True)
trello_table["LABELS"] = trello_table["LABELS"].str.replace(',', ' ', regex=True)
#  Temp. solution :)
trello_table['SEC_CONTROL'] = trello_table['LABELS'].str.extract(r'(\bSC\d{2}\b)')
trello_table['RES_CODE'] = trello_table['LABELS'].str.extract(r'(\bRC\d{2}\b)')
trello_table['DEPENDENT_ON'] = trello_table['LABELS'].str.extract(r'(\bCO\d{2}\b)')
trello_table['ROOT_CAUSE'] = trello_table['LABELS'].str.extract(r'(\bTB\d{2}\b)')
trello_table['PRIORITY'] = trello_table['LABELS'].str.extract(r'(\bPR\d{2}\b)')
# Load labels translation as a separate external file to keep the confidentiality of your log sources
labels_translation = pd.read_csv('./INPUT/labels_translation.csv', index_col=0, header=None, squeeze=True).to_dict()
trello_table = trello_table.replace(labels_translation)
trello_table.drop('LABELS', axis=1, inplace=True)


In [ ]:
mtta = round(trello_table['BUSINSESS_HOURS_TO_ACKNOWLEDGE'].mean(), 2) 
mttr = round(trello_table['BUSINSESS_HOURS_TO_RESOLVE'].mean(), 2) 
if DEBUG:    
    print('Mean time to resolve = ', mttr, 'business hours')
    print('Mean time to ack = ', mtta, 'business hours')

# 2. Data Visualization 

In [ ]:
required_fields = ['SEC_CONTROL', 'RES_CODE', 'DEPENDENT_ON', 'ROOT_CAUSE', 'PRIORITY']

trello_table.start_timestamp = trello_table['TICKET_CREATION_TIMESTAMP'].min().strftime('%d/%m/%Y|%H:%M:%S')
trello_table.end_timestamp = trello_table['TICKET_CREATION_TIMESTAMP'].max().strftime('%d/%m/%Y|%H:%M:%S')
trello_table.no_days = len(trello_table['TICKET_CREATION_TIMESTAMP'].dt.normalize().unique())

for required_field in required_fields:
    copy_of_trello_table = trello_table.copy()
    series = pd.value_counts(copy_of_trello_table[required_field])
    mask = (series/series.sum() * 100).lt(1.0)
    copy_of_trello_table[required_field] = np.where(copy_of_trello_table[required_field].isin(series[mask].index),'OTHERS ≤ 1.0%'.format(required_field),copy_of_trello_table[required_field])
    required_field_count = copy_of_trello_table[required_field].value_counts()    
    required_field_count = required_field_count.rename_axis(required_field).reset_index(name='{}_COUNT'.format(required_field))
    required_field_count = required_field_count.reset_index(drop=True)
    required_field_count.index.rename('NO.', inplace=True)
    required_field_count.index+=1
    required_field_count['{}_PCT'.format(required_field)] = required_field_count['{}_COUNT'.format(required_field)] / required_field_count['{}_COUNT'.format(required_field)].sum()
    # required_field_count.to_csv('./OUTPUT/{}.csv'.format(required_field), sep=',')
    fig = go.Figure(data=[go.Pie(labels=required_field_count[required_field], values=required_field_count['{}_PCT'.format(required_field)], textinfo='label+percent',
                             insidetextorientation='radial', showlegend=False,marker=dict(colors=colors, line=dict(color='#000000', width=2))
                            )])
    fig.update_layout(font_family=default_font_family)
    fig.update_layout(title='Security Investigation Tickets distributed by {}'.format(required_field))
    fig.layout.images = [dict( source=logo_path, xref='paper', yref='paper', x=0.95,  y=1, sizex=0.25, sizey=0.25, xanchor='center', yanchor='bottom')]
    fig.write_image("./OUTPUT/{}_COUNT.pdf".format(required_field))

    # fig.show()

In [ ]:
# Generate a summary table of the main features
summary_table = trello_table.groupby(['SEC_CONTROL', 'RES_CODE']).size().unstack(fill_value=0)
summary_table.to_csv('./OUTPUT/SUMMARY_TABLE.csv', sep=',')

In [ ]:
trello_table.start_timestamp = trello_table['TICKET_CREATION_TIMESTAMP'
        ].min().strftime('%d/%m/%Y|%H:%M:%S')
trello_table.end_timestamp = trello_table['TICKET_CREATION_TIMESTAMP'
        ].max().strftime('%d/%m/%Y|%H:%M:%S')
trello_table.no_days = len(trello_table['TICKET_CREATION_TIMESTAMP'
                           ].dt.normalize().unique())

tickets_count = trello_table['T#'].value_counts()
tickets_count = tickets_count.rename_axis('T#'
        ).reset_index(name='T#_COUNT')
tickets_count = tickets_count.reset_index(drop=True)
tickets_count.index.rename('NO.', inplace=True)
tickets_count.index += 1
tickets_count['T#_PCT'] = tickets_count['T#_COUNT'] \
    / tickets_count['T#_COUNT'].sum()
s = pd.to_datetime(trello_table['TICKET_CREATION_TIMESTAMP'])
tickets_count = s.groupby(s.dt.floor('d'
                          )).size().reset_index(name='COUNT')

# Plot ----------------------------------------------------------------------------------------------

my_layout = go.Layout(title='Global Font', font=dict(color='#7f7f7f',
                      size=10))
my_data = go.Scatter(x=tickets_count['TICKET_CREATION_TIMESTAMP'],
                     y=tickets_count['COUNT'], mode='lines+markers+text'
                     )
fig = go.Figure(data=my_data)

fig.update_layout(shapes=[go.layout.Shape(  # Line Horizontal
    type='line',
    x0=tickets_count['TICKET_CREATION_TIMESTAMP'].min(),
    y0=tickets_count['COUNT'].mean(),
    x1=tickets_count['TICKET_CREATION_TIMESTAMP'].max(),
    y1=tickets_count['COUNT'].mean(),
    line=dict(color='black', width=1, dash='longdash'),
    )])

fig.add_trace(go.Scatter(
    x=[tickets_count['TICKET_CREATION_TIMESTAMP'].max()
       - pd.Timedelta(days=trello_table.no_days) / 2.0],
    y=[tickets_count['COUNT'].mean()],
    mode='markers+text',
    name='Markers and Text',
    hoverinfo='skip',
    textposition='top right',
    ))

fig.update_traces(marker_color='rgb(231,198,91)',
                  marker_line_color='black', marker_line_width=1,
                  opacity=1.0)
fig.update_layout(title_text='VSOC tickets trendline grouped by the day<br>Dashed line represents the average no. VSOC tickets ({})'.format(int(tickets_count['COUNT'
                  ].mean())), font_size=default_font_size)

fig.update_layout(font_family=default_font_family, showlegend=False)
fig.layout.images = [dict(
    source=logo_path,
    x=0.9,
    y=1.05,
    sizex=0.25,
    sizey=0.25,
    xanchor='center',
    yanchor='bottom',
    )]
fig.update_layout(font_size=default_font_size)
fig.write_image('./OUTPUT/TRENDLINE.pdf')


In [ ]:
mbhtr_by_department = pd.DataFrame(columns = ['DEPENDENT_ON','MEAN_BUSINSESS_HOURS_TO_RESOLVE'])

for dept in trello_table['DEPENDENT_ON'].unique():
    test = trello_table[trello_table['DEPENDENT_ON'].str.contains(dept)]
    mbhtr_by_department = mbhtr_by_department.append({'DEPENDENT_ON': dept, 'MEAN_BUSINSESS_HOURS_TO_RESOLVE': round(test['BUSINSESS_HOURS_TO_RESOLVE'].mean(),2)}, ignore_index=True)


fig = px.bar(mbhtr_by_department, x='DEPENDENT_ON', y='MEAN_BUSINSESS_HOURS_TO_RESOLVE', text='MEAN_BUSINSESS_HOURS_TO_RESOLVE')
fig.update_traces(marker_color='#E7C65B', marker_line_color='black',
                  marker_line_width=1.5, opacity=1)
fig.layout.images = [dict( source=logo_path, xref='paper', yref='paper', x=0.95,  y=1, sizex=0.25, sizey=0.25, xanchor='center', yanchor='bottom')]

fig.update_layout(
    title="Mean Business Hours to Resolve By Department",
    xaxis_title='Dependency by department',
    yaxis_title='Total number of business hours',
    font=dict(
        family=default_font_family,
        size=default_font_size,
    )
)
fig.write_image('./OUTPUT/MEAN_BUSINSESS_HOURS_TO_RESOLVE_BY_DEPARTMENT.pdf')

In [ ]:
mbhtr_by_priority = pd.DataFrame(columns = ['PRIORITY','MEAN_BUSINSESS_HOURS_TO_RESOLVE'])


for prio in trello_table['PRIORITY'].unique():
    test = trello_table[trello_table['PRIORITY'].str.contains(prio)]
    mbhtr_by_priority = mbhtr_by_priority.append({'PRIORITY': prio, 'MEAN_BUSINSESS_HOURS_TO_RESOLVE': round(test['BUSINSESS_HOURS_TO_RESOLVE'].mean(),2)}, ignore_index=True)


fig = px.bar(mbhtr_by_priority, x='PRIORITY', y='MEAN_BUSINSESS_HOURS_TO_RESOLVE', text='MEAN_BUSINSESS_HOURS_TO_RESOLVE')
fig.update_traces(marker_color='#E7C65B', marker_line_color='black',
                  marker_line_width=1.5, opacity=1)
fig.layout.images = [dict( source=logo_path, xref='paper', yref='paper', x=0.95,  y=1, sizex=0.25, sizey=0.25, xanchor='center', yanchor='bottom')]

fig.update_layout(
    title="Mean Business Hours to Resolve By Priority",
    xaxis_title='Ticket Priority',
    yaxis_title='Total number of business hours',
    font=dict(
        family=default_font_family,
        size=default_font_size,
    )
)
fig.write_image('./OUTPUT/MEAN_BUSINSESS_HOURS_TO_RESOLVE_BY_PRIORITY.pdf')

In [ ]:
# #  FIGURE 1: EOI Chart
# ce_categories_center = ['caused by external actors', 'caused by internal technical actors', 'caused by internal non-technical actors']
# lasTICKET_month_ce_count = [33, 35, 12]
# currenTICKET_month_ce_count = [19, 22, 15]
# ce_sum = list(((np.array(currenTICKET_month_ce_count ) + np.array(lasTICKET_month_ce_count))))
# delta_ce = list(((np.array(currenTICKET_month_ce_count) - np.array(lasTICKET_month_ce_count )) / lasTICKET_month_ce_count ) * 100)
# delta_ce = np.round(delta_ce, decimals=1)

# green_tag = '<span style="color:green">▼</span>'
# red_tag = '<span style="color:red">▲</span>'
# logo_path = os.path.join(os.getcwd(), 'INPUT/sip-logo.png')

# fig = go.Figure()
# fig.add_trace(go.Bar(
#     x=ce_categories_center,
#     y=lasTICKET_month_ce_count,
#     name='SI count on August',
#     marker_color='rgb(231,198,91)', 
#     marker_line_color='black',
#     textposition='inside', 
#     text=lasTICKET_month_ce_count,
#     marker_line_width=1.5, 
#     opacity=1.0, width=0.5))

# fig.add_trace(go.Bar(
#     x=ce_categories_center,
#     y=currenTICKET_month_ce_count,
#     name='SI count in September', 
#     text=currenTICKET_month_ce_count,
#     textposition='inside',
#     marker_color='white', 
#     marker_line_color='black',
#     marker_line_width=1.5, 
#     opacity=0.6, width=0.6))

# fig.add_trace(go.Bar(
#     x=ce_categories_center, 
#     y=max(currenTICKET_month_ce_count,lasTICKET_month_ce_count),
#     text=delta_ce,
#     textposition='outside',
#     marker_line_width=1.5, 
#     opacity=0, 
#     width=0.6,
#     showlegend=False))

# for i, (x, y, z) in enumerate(zip(currenTICKET_month_ce_count, lasTICKET_month_ce_count, delta_ce)):
#     print(i, x, y, z)
#     if z > 0:
#         fig.add_annotation(x=i,y=max(x+1, y+1),text='<b>▲+{}%</b>'.format(z),showarrow=False,font=dict(color='red', size=12),align='center')
#     else:
#         fig.add_annotation(x=i,y=max(x+1, y+1),text='<b>▼{}%</b>'.format(z),showarrow=False,font=dict(color='green', size=12),align='center')


# title_plot1 = '<b> Increase {} and decrease {} in security incidents count categorized by root cause</b>'.format(red_tag, green_tag)

# fig.layout.images = [dict(
#     source=logo_path,
#     xref='paper', 
#     yref='paper',
#     x=0.05, 
#     y=1.05,
#     sizex=0.11, 
#     sizey=0.15,
#     xanchor='center', 
#     yanchor='bottom')]

# fig.update_layout(
#     height=600,
#     width=1200,
#     template='plotly_white',
#     font_family='IBM Plex Sans',
#     font_size=14,
#     barmode='overlay', 
#     xaxis_tickangle=0,
#     title=title_plot1,
#     title_x=0.5, 
#     font=dict(color='black'),
#     legend=dict(
#     orientation='h',
#     yanchor='top',
#     y=1.1,
#     xanchor='center',
#     x=0.5))
    
# fig.show()
# fig.write_image('./OUTPUT/EOI.pdf')

# 3. Reports Generation 

In [ ]:
duration_start = trello_table['TICKET_CREATION_TIMESTAMP'].min().strftime('%Y-%m-%d')
duration_end = trello_table['TICKET_CREATION_TIMESTAMP'].max().strftime('%Y-%m-%d')


# Save an analyst complete report
trello_table = trello_table.reset_index(drop=True)
trello_table.index+=1
file_name = '{}-{} SOC_REPORT.html'.format(duration_start, duration_end)
trello_table.to_html('./OUTPUT/{}'.format(file_name))
html_table_blue_light = build_table(trello_table, 'grey_light', font_family=default_font_family, index=True)
with open('./OUTPUT/{}'.format(file_name), 'w') as f:
    f.write(html_table_blue_light)


# Save a customer report
customer_report = trello_table[['T#', 'TICKET_CREATION_TIMESTAMP', 'PRIORITY','SEC_CONTROL', 'RES_CODE', 'DESC']]
file_name = '{}-{} CUS_SOC_REPORT.html'.format(duration_start, duration_end)
customer_report.to_html('./OUTPUT/{}'.format(file_name))
html_table_blue_light = build_table(customer_report, 'grey_light', font_family=default_font_family, index=True)
with open('./OUTPUT/{}'.format(file_name), 'w') as f:
    f.write(html_table_blue_light)

In [ ]:
fig = px.sunburst(trello_table, path=['PRIORITY', 'RES_CODE'], values='BUSINSESS_HOURS_TO_ACKNOWLEDGE',
                  color='BUSINSESS_HOURS_TO_ACKNOWLEDGE', hover_data=['BUSINSESS_HOURS_TO_ACKNOWLEDGE'],
                  color_continuous_midpoint=np.average(trello_table['BUSINSESS_HOURS_TO_ACKNOWLEDGE'], weights=trello_table['BUSINSESS_HOURS_TO_ACKNOWLEDGE']))
fig.show()